### SSP Trajectory Encoding

This demo illustrates how SSPs can be used to memorize and replay continuous trajectories. The "smoothness" of the replay depends on the number of sample points used to learn the trajectory, the dimensionality of the SSP, the scaling used to interpolate between specific sample points, and distance between the points used to "cue" each sample in the stored trajectory. Relationships between these variables are still being investigated, but it seems like different configurations of them might be used to achieve the same effect, indicating a certain degree of redundancy.

For example, the `scale` argument is used to scale the exponentiation that encodes grid points over the represented plane - roughly, the larger this argument is, the more dissimilar vector representations of adjacent points are to one another. When points are encoded into a trajectory by binding each point into a position in a continuous 1D "cue" space, we can retrieve a given point from the encoding by binding with the inverse of the vector representation of the right cue space position. If we move between two positions in this cue space, we'll extract a mix of the points bound to these positions, and the degree of the mix will depend on how far we are from each position. Then, how this mix of points translates into interpolation on the grid mentioned above depends on the scale argument, since with a large scale value, there will almost no similarity between the representation of one point and any others.

One consequence of this reasoning is that it's actually not clear that we're interpolating in a useful way here, since when we decrease the scale, we're effectively lowering the resolution of the spatial map while including the same number of encoded points in each trajectory. These points then better cover the entire span of the lower-resolution trajectory. Switching the scale between 5 and 20 and running the notebook will illustrate.

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import HTML
import numpy as np

from ssp.maps import Spatial2D
from ssp.utils import interpolate, linear_steps
from ssp.plots import heatmap_animation, create_gif

In [ ]:
dim = 1024
scale = 5
traj_steps = 25
step_size = 1

x_len = 2
y_len = 2
x_spaces = 101
y_spaces = 101

In [ ]:
ssp_map = Spatial2D(dim=dim, scale=scale)
ssp_map.build_grid(x_len, y_len, x_spaces, y_spaces)

In [ ]:
# define some functions to use for generating trajectories
absval = lambda x: np.abs(x)
cosine = lambda x: 0.5 * np.cos(x * np.pi) + 1
circle = lambda x, n: (1 + np.cos(2 * np.pi / n * x) / 2, 1 + np.sin(2 * np.pi / n * x) / 2)

absval_points = [(x, absval(x)) for x in np.linspace(0, ssp_map.x_len, traj_steps)]
cosine_points = [(x, cosine(x)) for x in np.linspace(0, ssp_map.x_len, traj_steps)]
circle_points = [circle(x, traj_steps) for x in range(traj_steps)]

In [ ]:
# encode and decode the trajectories as defined by sample points
enc_cues = linear_steps(traj_steps, stepsize=step_size)
dec_cues = interpolate(enc_cues, n=5)

enc = ssp_map.encode_trajectory(points=absval_points, cues=enc_cues)
dec = ssp_map.decode_trajectory(enc, cues=dec_cues)
absval_sims = [ssp_map.compute_heatmap(v) for v in dec]

enc = ssp_map.encode_trajectory(points=cosine_points, cues=enc_cues)
dec = ssp_map.decode_trajectory(enc, cues=dec_cues)
cosine_sims = [ssp_map.compute_heatmap(v) for v in dec]

enc = ssp_map.encode_trajectory(points=circle_points, cues=enc_cues)
dec = ssp_map.decode_trajectory(enc, cues=dec_cues)
circle_sims = [ssp_map.compute_heatmap(v) for v in dec]

In [ ]:
# plot the results
n_plots = 3
n_steps = len(dec_cues)
figsize = (16, 8)
sims = [absval_sims, cosine_sims, circle_sims]

ani = heatmap_animation(sims, figsize=figsize)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani)))